In [1]:
import itertools
import pandas as pd
from scipy import stats
from odapi.connectors import Irceline
from odapi.settings import settings
settings.logger.setLevel(50)

In [2]:
client = Irceline()

In [3]:
client.meta.columns

Index(['serieid', 'siteid', 'measureid', 'serieunits', 'measurekey',
       'measurename', 'sitekey', 'sitename', 'seriekey', 'molarmass', 'factor',
       'sitelocation', 'sitetype', 'lat', 'lon', 'nuts1id', 'nuts2id',
       'nuts3id', 'nuts1name', 'nuts2name', 'nuts3name', 'lauid', 'launame',
       'started', 'stopped'],
      dtype='object')

In [4]:
sel = client.select(measurekey='NO2', sitekey='41')[["serieid", "sitekey", "measurekey"]]

In [5]:
sel.sample(2)

,serieid,sitekey,measurekey
99,6528,41B011,NO2
100,6551,41MEU1,NO2


In [6]:
t0 = "2019-01-01"
t1 = "2020-01-01"
t2 = "2019-10-01"

In [7]:
recs = client.get_records(sel, start=t0, stop=t1)

In [8]:
recs.sample(2)

,serieid,start,stop,value
3098,6639,2019-05-10 01:00:00+00:00,2019-05-10 02:00:00+00:00,9.0
1529,6574,2019-03-05 16:00:00+00:00,2019-03-05 17:00:00+00:00,32.0


In [9]:
data = recs.merge(sel).pivot_table(index='start', columns='sitekey', values='value')

In [10]:
data.sample(5)

sitekey,41B001,41B004,41B006,41B008,41B011,41MEU1,41N043,41R001,41R002,41R012,41WOL1
start,,,,,,,,,,,
2019-02-05 01:00:00+00:00,33.5,36.5,2.5,17.5,6.5,25.0,20.5,9.5,11.5,3.5,9.5
2019-01-06 01:00:00+00:00,31.0,21.5,25.5,26.0,6.5,12.0,14.0,16.0,28.5,6.5,12.5
2019-08-19 12:00:00+00:00,42.0,15.0,NaN,NaN,4.0,16.0,33.5,18.5,33.0,8.5,17.5
2019-02-18 10:00:00+00:00,89.0,70.5,45.0,74.0,34.0,66.5,76.0,55.5,63.0,27.5,44.5
2019-11-27 22:00:00+00:00,24.0,6.0,6.0,19.0,0.0,8.5,8.5,4.0,8.5,3.5,4.0


In [11]:
def t_test(ref, exp, **params):
    res = stats.ttest_ind(ref, exp, equal_var=False, nan_policy='omit')
    rep = {k: getattr(res, k) for k in res._fields}
    rep["class"] = res.__class__.__name__
    rep["params"] = params
    return rep

In [12]:
def ks_test(ref, exp, **params):
    res = stats.ks_2samp(ref, exp, **params)
    rep = {k: getattr(res, k) for k in res._fields}
    rep["class"] = res.__class__.__name__
    rep["params"] = params
    return rep

In [13]:
def dispatch(ref, exp, callback, mode='product', **params):
    if mode == 'product':
        for x, y in itertools.product(ref, exp):
            res = callback(ref[x], exp[y], **params)
            res.update({'ref_key': x, 'exp_key': y})
            yield res
    elif mode == 'pairwise':
        for x in set(ref.columns).intersection(set(exp.columns)):
            res = callback(ref[x], exp[x], **params)
            res.update({'ref_key': x, 'exp_key': x})
            yield res

In [14]:
def apply_test(ref, exp, test_func=t_test, mode='product', **params):
    return pd.DataFrame([r for r in dispatch(ref, exp, callback=test_func, mode=mode, **params)])

In [15]:
ref = data.loc[:,['41R002', '41R012']]
exp = data.loc[t2:t1,:]

In [23]:
m = 'product'
t = apply_test(ref, exp, test_func=t_test, mode=m)
ks = apply_test(ref, exp, test_func=ks_test, mode=m)

In [24]:
test = pd.concat([t, ks])

In [25]:
test

,statistic,pvalue,class,params,ref_key,exp_key
0,-30.857060,1.850102e-185,Ttest_indResult,{},41R002,41B001
1,4.543881,5.708183e-06,Ttest_indResult,{},41R002,41B004
2,24.070985,3.157105e-119,Ttest_indResult,{},41R002,41B006
3,-23.870924,3.499832e-116,Ttest_indResult,{},41R002,41B008
4,34.512547,4.077110e-227,Ttest_indResult,{},41R002,41B011
5,19.398188,1.302347e-80,Ttest_indResult,{},41R002,41MEU1
6,-4.691089,2.823173e-06,Ttest_indResult,{},41R002,41N043
7,10.273145,1.932855e-24,Ttest_indResult,{},41R002,41R001
8,1.250522,2.111897e-01,Ttest_indResult,{},41R002,41R002
9,56.073081,0.000000e+00,Ttest_indResult,{},41R002,41R012


In [26]:
s = test.pivot_table(index='exp_key', columns=['class', 'ref_key'], values=["pvalue", "statistic"])

In [27]:
s = s.reorder_levels([2,1,0], axis=1).sort_index(axis=1)

In [28]:
s

ref_key         41R002                                              41R012  \
class   Ks_2sampResult           Ttest_indResult            Ks_2sampResult   
                pvalue statistic          pvalue  statistic         pvalue   
exp_key                                                                      
41B001             1.0  0.297176   1.850102e-185 -30.857060   0.000000e+00   
41B004             1.0  0.062928    5.708183e-06   4.543881  2.700991e-272   
41B006             1.0  0.219370   3.157105e-119  24.070985   1.000000e+00   
41B008             1.0  0.258664   3.499832e-116 -23.870924   0.000000e+00   
41B011             1.0  0.346192   4.077110e-227  34.512547   1.000000e+00   
41MEU1             1.0  0.182582    1.302347e-80  19.398188   1.000000e+00   
41N043             1.0  0.053374    2.823173e-06  -4.691089   0.000000e+00   
41R001             1.0  0.117919    1.932855e-24  10.273145  1.190893e-220   
41R002             1.0  0.054077    2.111897e-01   1.250522   0.000000e+00   
41R012             0.0  0.467471    0.000000e+00  56.073081   1.000000e+00   
41WOL1             1.0  0.121101    6.963416e-40  13.385058   1.000000e+00   

ref_key                                       
class             Ttest_indResult             
        statistic          pvalue  statistic  
exp_key                                       
41B001   0.651889    0.000000e+00 -76.424326  
41B004   0.421289   6.280976e-271 -39.524806  
41B006   0.244790    1.184674e-98 -21.932094  
41B008   0.602412    0.000000e+00 -65.671792  
41B011   0.119164    1.112698e-20  -9.396097  
41MEU1   0.350761   2.335742e-207 -33.104862  
41N043   0.466798    0.000000e+00 -46.966817  
41R001   0.379170   2.424245e-233 -35.840144  
41R002   0.468391    0.000000e+00 -44.955180  
41R012   0.056212    1.898931e-02   2.346787  
41WOL1   0.327693   1.251230e-170 -29.942836